<a href="https://colab.research.google.com/github/quangnguyen-james/Datamining-Technical/blob/main/Instacart_apriori_and_fbgrowth.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tổng quan dataset: **Instacart Market Basket Analysis**
* Bộ dataset về các giao dịch trong siêu thị Instacart
* Là bộ dữ liệu thực tế gồm hơn 3 triệu đơn hàng của khoảng 206 nghìn người dùng Instacart
* Mỗi đơn hàng gồm danh sách sản phẩm, kèm theo thời gian đặt (order_dow, order_hour_of_day), thứ tự đơn hàng và thông tin đã đặt lại (reorder)
* Mục đích: phân tích hành vi mua hàng để dự đoán sản phẩm người dùng sẽ tái đặt, khai phá luật kết hợp, và tối ưu cross-selling/up-selling

## Các file chính trong dataset
**Theo tài liệu và notebook tham khảo, bộ dữ liệu gồm 6 file CSV chính:**
  *  **aisles.csv:** Lưu trữ phân loại sản phẩm theo khu vực trưng bày, gồm:
     - *aisle_id*: ID lối kệ
     - *aisle*: tên lối kệ
  *  **departments.csv**: Lưu trữ danh mục nhóm sản phẩm, gồm:
     - *department_id*: id ngành hàng
     - *department*: tên nghành hàng
  *  **order_products__prior.csv:** Liệt kê sản phẩm đã mua trong các đơn thuộc tập "prior" (các đơn để rèn feature và lịch sử mua), gồm:
     - *order_id*: ID đơn hàng
     - *product_id*: ID sản phẩm
     - *add_to_cart_order*: thứ tự thêm vào giỏ
     - *reordered*: 1 nếu user đã mua món này trước đó, 0 ngược lại
  *  **order_products__train.csv:**  Tương tự prior, nhưng dùng để huấn luyện mô hình dự đoán “reorder” trong tập train (test chưa có reorder)
  *  **orders.csv:** Lưu trữ thông tin đơn hàng, gồm:
     - *order_id*: ID đơn hàng
     - *user_id*: ID người dùng
     - *eval_set*: thuộc “prior”, “train” hay “test”—phân chia tập để huấn luyện/dự đoán
     - *order_number*: thứ tự đơn hàng của mỗi user
     - *order_dow,order_hour_of_day*: ngày/giờ đơn
     - *days_since_prior_order*: số ngày kể từ đơn trước
  *  **products.csv:** Lưu trữ danh mục sản phẩm, gồm:
     - *product_id*: id sản phẩm
     - *product_name*: tên sản phẩm
     - *aisle_id,department_id*: kết nối với 2 bảng aisles và departments

## Ý nghĩa từng file
* **aisles.csv & departments.csv** → hỗ trợ phân loại sản phẩm, dùng trong phân tích theo nhóm, gợi ý sản phẩm, cross-selling.
* **products.csv** → danh mục sản phẩm, liên kết với đơn và phân nhóm.
* **orders.csv** → thông tin lịch sử mua hàng từng user (thời gian, khoảng cách giữa các đơn, tập prior/train/test).
* **order_products__prior.csv & order_products__train.csv** → thông tin chi tiết từng đơn hàng, thiết lập feature để dự đoán xem user có mua lại sản phẩm hay không.

# Chuẩn bị dataset
* Upload nguyên thư mục **Instacart Market Basket Analysis** (bao gồm 6 files) vào trong input
* Tiến hành đọc từng file (cần thiết)
****
**Phương pháp tiếp cận Dataset**
* Vì dataset quá lớn nên việc upload lên Colab cũng gây nhiều khó khăn; hiện Dataset này có sẵn trên Kaggle, ta có thể kết nối với Kaggle để đọc dữ liệu.
* Việc đọc Dataset từ Colab cần phải tạo môi trường kết nối (không thể đọc trực tiếp từ https)
* Các bước thiết lập môi trường kết nối:
  *  Bước 1: Cài đặt Kaggle API
  * Bước 2: Tải file kaggle.json (API token)
  * Bước 3: Tải file kaggle.json lên Colab
  * Bước 4: Thiết lập cấu hình cho kaggle
  * Bước 5: Tải dataset từ Kaggle
  * Bước 6: Giải nén & Sử dụng

In [1]:
import os
import zipfile
# Bước 1: Cài đặt Kaggle API
!pip install -q kaggle

# Bước 2: Tạo file kaggle.json (API token) từ Kaggle

# Bước 3: Tải file kaggle.json lên Colab
from google.colab import files
files.upload()

# Bước 4: Thiết lập cấu hình cho kaggle
# Tạo thư mục .kaggle và chép file vào
os.makedirs('/root/.kaggle', exist_ok=True)
!mv kaggle.json /root/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json

#  Bước 5: Tải dataset từ Kaggle
!kaggle datasets download -d psparks/instacart-market-basket-analysis

# Bước 6: Giải nén & SỬ dụng
!unzip -q instacart-market-basket-analysis.zip

# Sử dụng
import os
for dirname, _, filenames in os.walk('.'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

Saving kaggle.json to kaggle.json
Dataset URL: https://www.kaggle.com/datasets/psparks/instacart-market-basket-analysis
License(s): CC0-1.0
100% 197M/197M [00:00<00:00, 529MB/s]  
100% 197M/197M [00:00<00:00, 570MB/s]
./instacart-market-basket-analysis.zip
./products.csv
./order_products__prior.csv
./departments.csv
./orders.csv
./order_products__train.csv
./aisles.csv
./.config/default_configs.db
./.config/.last_update_check.json
./.config/active_config
./.config/hidden_gcloud_config_universe_descriptor_data_cache_configs.db
./.config/.last_survey_prompt.yaml
./.config/config_sentinel
./.config/gce
./.config/.last_opt_in_prompt.yaml
./.config/logs/2025.06.24/13.38.10.852366.log
./.config/logs/2025.06.24/13.37.31.578035.log
./.config/logs/2025.06.24/13.38.02.464816.log
./.config/logs/2025.06.24/13.38.01.317571.log
./.config/logs/2025.06.24/13.38.11.549981.log
./.config/logs/2025.06.24/13.37.52.425029.log
./.config/configurations/config_default
./sample_data/anscombe.json
./sample_data/

# Triển khai thuật toán luật kết hợp Apriori và FB-Growth

## Import thư viên, Đọc Dataset và các tập dữ liệu liên quan

In [1]:
import pandas as pd
import numpy as np
import os
from mlxtend.frequent_patterns import fpgrowth, apriori, association_rules
from mlxtend.preprocessing import TransactionEncoder
import random

products_data = pd.read_csv('/content/products.csv')
aisles_data = pd.read_csv('/content/aisles.csv')
departments_data = pd.read_csv('/content/departments.csv')
orders_data = pd.read_csv('/content/orders.csv')
order_products_prior_data = pd.read_csv('/content/order_products__prior.csv')
order_products_train_data = pd.read_csv('/content/order_products__train.csv')
products_data.info()
aisles_data.info()
departments_data.info()
orders_data.info()
order_products_prior_data.info()
order_products_train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49688 entries, 0 to 49687
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   product_id     49688 non-null  int64 
 1   product_name   49688 non-null  object
 2   aisle_id       49688 non-null  int64 
 3   department_id  49688 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 1.5+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 134 entries, 0 to 133
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   aisle_id  134 non-null    int64 
 1   aisle     134 non-null    object
dtypes: int64(1), object(1)
memory usage: 2.2+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21 entries, 0 to 20
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   department_id  21 non-null     int64 
 1   department     21 non-null     object
dtyp

## Tiền xử lý dữ liệu
### Gộp thông tin đầy đủ của Product từ aisles và departments
### Tạo 1 file order_detail từ việc gộp thông tin toàn bộ các files, cụ thể:
* File gồm các thông tin: order_id, product_id, product_name, aisle_id, aisle, department_id, department
* Nếu eval_set = "prior" thì lấy thông tin từ file order_products__prior
* Nếu eval_set = "train" thì lấy thông tin từ file order_products__train
* aisle và department được map từ dữ liệu product_data

In [2]:
# 1. Gộp thông tin đầy đủ của Product từ aisles và departments
products_data = pd.merge(products_data, aisles_data, on='aisle_id', how='left')
products_data = pd.merge(products_data, departments_data, on='department_id', how='left')
products_data.head()

# Tạo 1 file order_detail từ việc gộp thông tin toàn bộ các files, cụ thể:
# File gồm các thông tin: order_id, product_id, product_name, aisle_id, aisle, department_id, department
# Nếu eval_set = "prior" thì lấy thông tin từ file order_products__prior
# Nếu eval_set = "train" thì lấy thông tin từ file order_products__train
# aisle và department được map từ dữ liệu product_data

# 2. Chia orders thành prior và train
orders_prior = orders_data[orders_data['eval_set'] == 'prior']
orders_train = orders_data[orders_data['eval_set'] == 'train']

# 3. Kết hợp thông tin từ order_products và sản phẩm
prior_detail = pd.merge(order_products_prior_data, products_data, on='product_id', how='left')
train_detail = pd.merge(order_products_train_data, products_data, on='product_id', how='left')

# 4. Gộp thông tin orders (nếu cần thêm user_id v.v.)
prior_detail = pd.merge(prior_detail, orders_prior[['order_id']], on='order_id', how='inner')
train_detail = pd.merge(train_detail, orders_train[['order_id']], on='order_id', how='inner')

# 5. Gộp cả 2 vào một bảng chung
order_detail = pd.concat([prior_detail, train_detail], ignore_index=True)

# 6. Giữ lại các cột như định nghĩa (định nghĩa process_data)
process_data = order_detail[['order_id', 'product_id', 'product_name', 'aisle_id', 'aisle', 'department_id', 'department']]

process_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33819106 entries, 0 to 33819105
Data columns (total 7 columns):
 #   Column         Dtype 
---  ------         ----- 
 0   order_id       int64 
 1   product_id     int64 
 2   product_name   object
 3   aisle_id       int64 
 4   aisle          object
 5   department_id  int64 
 6   department     object
dtypes: int64(4), object(3)
memory usage: 1.8+ GB


In [5]:
#Kiểm tra trùng
print(process_data.duplicated(subset=['order_id', 'product_id']).sum())

# # Xóa dữ liệu trùng (nếu số dữ liệu trùng tồn tại)
# process_data = process_data.drop_duplicates()

# Số sample còn lại
print(process_data.shape)

0


In [6]:
process_data.head(50)

,order_id,product_id,product_name,aisle_id,aisle,department_id,department
0,2,33120,Organic Egg Whites,86,eggs,16,dairy eggs
1,2,28985,Michigan Organic Kale,83,fresh vegetables,4,produce
2,2,9327,Garlic Powder,104,spices seasonings,13,pantry
3,2,45918,Coconut Butter,19,oils vinegars,13,pantry
4,2,30035,Natural Sweetener,17,baking ingredients,13,pantry
5,2,17794,Carrots,83,fresh vegetables,4,produce
6,2,40141,Original Unflavored Gelatine Mix,105,doughs gelatins bake mixes,13,pantry
7,2,1819,All Natural No Stir Creamy Almond Butter,88,spreads,13,pantry
8,2,43668,Classic Blend Cole Slaw,123,packaged vegetables fruits,4,produce
9,3,33754,Total 2% with Strawberry Lowfat Greek Strained...,120,yogurt,16,dairy eggs


## Bài toán phân tích sự kết hợp của việc xếp hàng lên kệ

In [7]:
#Gộp Items trong cùng 1 giao dịch
aisle_transactions = process_data.groupby('order_id')['aisle'].apply(list)
print(aisle_transactions)


order_id
1          [yogurt, other creams cheeses, fresh vegetable...
2          [eggs, fresh vegetables, spices seasonings, oi...
3          [yogurt, soy lactosefree, packaged vegetables ...
4          [breakfast bakery, cold flu allergy, energy gr...
5          [fresh fruits, salad dressing toppings, prepar...
                                 ...                        
3421079                                               [soap]
3421080    [milk, ice cream ice, juice nectars, fresh veg...
3421081    [chips pretzels, bread, condiments, packaged c...
3421082    [packaged poultry, butter, fresh fruits, packa...
3421083    [nuts seeds dried fruit, baby food formula, ba...
Name: aisle, Length: 3346083, dtype: object


In [8]:
# 2. Chuyển dữ liệu sang định dạng one-hot encoding
te = TransactionEncoder()
te_ary = te.fit(aisle_transactions).transform(aisle_transactions)
df = pd.DataFrame(te_ary, columns=te.columns_)
print("\nDataFrame one-hot encoding:")
print(df)


DataFrame one-hot encoding:
         air fresheners candles  asian foods  baby accessories  \
0                         False        False             False   
1                         False        False             False   
2                         False        False             False   
3                         False        False             False   
4                         False        False             False   
...                         ...          ...               ...   
3346078                   False        False             False   
3346079                   False        False             False   
3346080                   False        False             False   
3346081                   False        False             False   
3346082                   False        False             False   

         baby bath body care  baby food formula  bakery desserts  \
0                      False              False            False   
1                      False              

## Thuật toán Apriori (Vì dữ liệu quá lớn, nên không thể triển khai thuật toán này)

In [10]:
minSupport = 0.01
minConfidence = 0.1

In [ ]:
# # 3. Chạy thuật toán Apriori
# frequent_itemsets = apriori(df, min_support=minSupport, use_colnames=True)
# print("Tập mục phổ biến:")
# print(frequent_itemsets)
# # 4. Sinh luật kết hợp
# rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=minConfidence)
# print("\nLuật kết hợp:")
# print(rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']])

Tập mục phổ biến:
      support                       itemsets
0    0.036344                    (Alfajores)
1    0.016059                     (Baguette)
2    0.005071                     (Bakewell)
3    0.327205                        (Bread)
4    0.040042                      (Brownie)
..        ...                            ...
109  0.007184      (Coffee, Bread, Sandwich)
110  0.007396           (Coffee, Bread, Tea)
111  0.006867  (Coffee, Hot chocolate, Cake)
112  0.010037            (Coffee, Cake, Tea)
113  0.005388        (Coffee, Tea, Sandwich)

[114 rows x 2 columns]

Luật kết hợp:
            antecedents consequents   support  confidence      lift
0           (Alfajores)     (Bread)  0.010354    0.284884  0.870657
1           (Alfajores)    (Coffee)  0.019651    0.540698  1.130235
2           (Alfajores)       (Tea)  0.006762    0.186047  1.304393
3             (Brownie)     (Bread)  0.010777    0.269129  0.822508
4                (Cake)     (Bread)  0.023349    0.224822  0.68

## Thuật toán FBGrowth

In [ ]:
frequent_itemsets = fpgrowth(df, min_support=minSupport, use_colnames=True)
print("Tập mục phổ biến:")
print(frequent_itemsets)
# 4. Tạo luật kết hợp
min_confidence = 0.7  # Ngưỡng confidence tối thiểu:
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=minConfidence)
# 5. In kết quả
print("Frequent Itemsets:")
print(frequent_itemsets.sort_values(by='support', ascending=False).head(10))
print("\nAssociation Rules:")
print(rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']].head(10))